# Retrieving Result Collections

This example shows how QCSubmit can be used to retrieve the results of quantum chemical (QC) calculations from a [QCFractal] instance such as [QCArchive].

In particular, it demonstrates how:

* raw torsion drive, optimised geometry and hessian result records can be retrieved from the public
  [QCArchive] server and stored in a result collection

* the retrieved result records can be filtered and curated using a set of built-in filters

* the result collection can be saved and loaded from disk

[QCFractal]: http://docs.qcarchive.molssi.org/projects/qcfractal/en/latest/
[QCArchive]: https://qcarchive.molssi.org/

For the sake of clarity all verbose warnings will be disabled in this tutorial:

In [1]:
import warnings

warnings.filterwarnings("ignore")
import logging

logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

## Retrieving result collections

QCSubmit provides a suite of utilities for retrieving and curating collections of QC results directly from a running QCFractal server, or an already computed QCPortal dataset. This functionality is provided through three main classes:

* `BasicResultCollection` - stores references to simple QCPortal result record that may contain energies, gradients, or hessians computed for a molecule in a single conformation.

* `OptimizationResultCollection` - stores references to full optimization result records (i.e. `OptimizationRecord`
  objects), as well as the final minimised conformer produced by the optimization.

* `TorsionDriveResultCollection` - stores references to full torsion drive result records (i.e. `TorsionDriveRecord`
  objects), as well as the minimum energy conformer associated with each torsion angle that was scanned.

Each of these collections can be generated directly from a running `QCFractal` server using the `from_server` class
method.

We begin by creating a QCPortal `PortalClient` instance that will allow us to communicate with the running
server at `https://api.qcarchive.molssi.org:443`. We also supply the `cache_dir` argument as the current directory,
which will lead to each downloaded dataset being saved to a SQLite database.

In [2]:
from qcportal import PortalClient

qc_client = PortalClient("https://api.qcarchive.molssi.org:443", cache_dir=".")

Many of the datasets we want to retrieve will be quite large, so we also define a
small helper function to print only a subset of their records:

In [3]:
from copy import deepcopy


def preview_dataset(ds, max_entries=5):
    ds = deepcopy(ds)
    for key, entry in ds.entries.items():
        ds.entries[key] = entry[:max_entries]
    print(ds)

We can then use our client to generate our result collections:

In [4]:
from openff.qcsubmit.results import (
    BasicResultCollection,
    OptimizationResultCollection,
    TorsionDriveResultCollection,
)

# Pull down the energy result records from the 'OpenFF BCC Refit Study COH v1.0' dataset.
energy_result_collection = BasicResultCollection.from_server(
    client=qc_client,
    datasets="OpenFF BCC Refit Study COH v1.0",
    spec_name="spec_2",  # This used to be "resp-2-vacuum", but the spec name was changed in the QCArchive 0.50 migration
)
preview_dataset(energy_result_collection)

# Pull down the optimization records from both the 'OpenFF Gen 2 Opt Set 3 Pfizer Discrepancy' and
# 'OpenFF Gen 2 Opt Set 4 eMolecules Discrepancy' datasets.
optimization_result_collection = OptimizationResultCollection.from_server(
    client=qc_client,
    datasets=[
        "OpenFF Gen 2 Opt Set 3 Pfizer Discrepancy",
        "OpenFF Gen 2 Opt Set 4 eMolecules Discrepancy",
    ],
    spec_name="default",
)
preview_dataset(optimization_result_collection)

# Pull down the torsion drive records from the 'OpenFF Protein Capped 3-mer Backbones v1.0' dataset.
torsion_drive_result_collection = TorsionDriveResultCollection.from_server(
    client=qc_client,
    datasets="OpenFF Protein Capped 3-mer Backbones v1.0",
    spec_name="default",
)
preview_dataset(torsion_drive_result_collection)

entries={'https://api.qcarchive.molssi.org:443/': [BasicResult(type='basic', record_id=32651764, cmiles='[H:6][C:1]1([C:2]([C:4]([C:5]([C:3]1([H:10])[H:11])([H:14])[H:15])([H:12])[H:13])([H:8])[H:9])[H:7]', inchi_key='RGSFGYAAUTVSQA-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651722, cmiles='[H:7][c:1]1[c:2]([c:4]([c:6]([c:5]([c:3]1[H:9])[H:11])[H:12])[H:10])[H:8]', inchi_key='UHOVQNZJYSORNB-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651734, cmiles='[H:7][C:1]1([C:2]([C:4]([C:6]([C:5]([C:3]1([H:11])[H:12])([H:15])[H:16])([H:17])[H:18])([H:13])[H:14])([H:9])[H:10])[H:8]', inchi_key='XDTMQSROBMDMFD-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651809, cmiles='[H:11][c:1]1[c:2]([c:4]([c:6]([c:5]([c:3]1[H:13])[H:15])[C:7]([H:16])([H:17])[C:8]([H:18])([H:19])[C:9]([H:20])([H:21])[O:10][H:22])[H:14])[H:12]', inchi_key='VAJVDSVGBWFCLW-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651781, cmiles='[H:11][c:1]1[c:2]([c:4]([c:6]([c:5]([c:3]1[H:13])[H:15])[C:7]

entries={'https://api.qcarchive.molssi.org:443/': [OptimizationResult(type='optimization', record_id=6091201, cmiles='[H:18][c:1]1[c:2]([c:6]([c:10]2[c:9]([c:5]1[H:22])[C@:14]([C:12]([C:13]([O:17]2)([H:28])[H:29])([H:26])[H:27])([H:30])[N:16]([H:31])[c:11]3[c:7]([c:3]([c:4]([c:8]([n:15]3)[H:25])[H:21])[H:20])[H:24])[H:23])[H:19]', inchi_key='MYKMQNNSQCAQMH-MECQSIQINA-N'), OptimizationResult(type='optimization', record_id=6091202, cmiles='[H:18][c:1]1[c:2]([c:6]([c:10]2[c:9]([c:5]1[H:22])[C@:14]([C:12]([C:13]([O:17]2)([H:28])[H:29])([H:26])[H:27])([H:30])[N:16]([H:31])[c:11]3[c:7]([c:3]([c:4]([c:8]([n:15]3)[H:25])[H:21])[H:20])[H:24])[H:23])[H:19]', inchi_key='MYKMQNNSQCAQMH-MECQSIQINA-N'), OptimizationResult(type='optimization', record_id=6091203, cmiles='[H:18][c:1]1[c:2]([c:6]([c:10]2[c:9]([c:5]1[H:22])[C@:14]([C:12]([C:13]([O:17]2)([H:28])[H:29])([H:26])[H:27])([H:30])[N:16]([H:31])[c:11]3[c:7]([c:3]([c:4]([c:8]([n:15]3)[H:25])[H:21])[H:20])[H:24])[H:23])[H:19]', inchi_key='MYKMQNNS

entries={'https://api.qcarchive.molssi.org:443/': [TorsionDriveResult(type='torsion', record_id=104348544, cmiles='[H:13][C@@:8]([C:9](=[O:10])[N:17]([H:25])[C@:18]([H:26])([C:19](=[O:20])[N:30]([H:35])[C@:31]([H:36])([C:32](=[O:33])[N:40]([H:42])[C:41]([H:43])([H:44])[H:45])[C:34]([H:37])([H:38])[H:39])[C:21]([H:27])([H:28])[C:22](=[O:23])[O:24][H:29])([C:11]([H:14])([H:15])[H:16])[N:7]([H:12])[C:1](=[O:2])[C:3]([H:4])([H:5])[H:6]', inchi_key='DFRSMLTYXUADOY-WMXHRXQGNA-N'), TorsionDriveResult(type='torsion', record_id=104348638, cmiles='[H:13][C@@:8]([C:9](=[O:10])[N:17]([H:25])[C@:18]([H:26])([C:19](=[O:20])[N:31]([H:36])[C@:32]([H:37])([C:33](=[O:34])[N:41]([H:43])[C:42]([H:44])([H:45])[H:46])[C:35]([H:38])([H:39])[H:40])[C:21]([H:27])([H:28])[C:22](=[O:23])[N:24]([H:29])[H:30])([C:11]([H:14])([H:15])[H:16])[N:7]([H:12])[C:1](=[O:2])[C:3]([H:4])([H:5])[H:6]', inchi_key='KYQIKKWSTMGEQA-RIYGDZFYNA-N'), TorsionDriveResult(type='torsion', record_id=104348640, cmiles='[H:13][C@@:8]([C:9]

*Note: currently only complete results are pulled down by the `from_server` method*

There are two main inputs to the `from_server` method, in addition to the fractal client:

* the name(s) of the existing datasets to retrieve the results of. This can either be the name of a single dataset or a list of dataset names
* the name of the specification used to compute the records. Each specification corresponds to a particular basis, method, program and additional settings.

Let's print out some basic information about each of these result collections:

In [5]:
print("===HESSIAN RESULTS===")

print(f"N RESULTS:   {energy_result_collection.n_results}")
print(f"N MOLECULES: {energy_result_collection.n_molecules}")

print("===OPTIMIZATION RESULTS===")

print(f"N RESULTS:   {optimization_result_collection.n_results}")
print(f"N MOLECULES: {optimization_result_collection.n_molecules}")

print("===TORSION DRIVE RESULTS===")

print(f"N RESULTS:   {torsion_drive_result_collection.n_results}")
print(f"N MOLECULES: {torsion_drive_result_collection.n_molecules}")

===HESSIAN RESULTS===
N RESULTS:   191
N MOLECULES: 91
===OPTIMIZATION RESULTS===
N RESULTS:   2398
N MOLECULES: 419
===TORSION DRIVE RESULTS===
N RESULTS:   23
N MOLECULES: 23


We can easily save / load the collections to / from disk:

In [6]:
# save the energy result collection to a JSON file
with open("energy-result-collection.json", "w") as file:
    file.write(energy_result_collection.json())

# re-load the serialized result collection
preview_dataset(BasicResultCollection.parse_file("energy-result-collection.json"))

entries={'https://api.qcarchive.molssi.org:443/': [BasicResult(type='basic', record_id=32651764, cmiles='[H:6][C:1]1([C:2]([C:4]([C:5]([C:3]1([H:10])[H:11])([H:14])[H:15])([H:12])[H:13])([H:8])[H:9])[H:7]', inchi_key='RGSFGYAAUTVSQA-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651722, cmiles='[H:7][c:1]1[c:2]([c:4]([c:6]([c:5]([c:3]1[H:9])[H:11])[H:12])[H:10])[H:8]', inchi_key='UHOVQNZJYSORNB-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651734, cmiles='[H:7][C:1]1([C:2]([C:4]([C:6]([C:5]([C:3]1([H:11])[H:12])([H:15])[H:16])([H:17])[H:18])([H:13])[H:14])([H:9])[H:10])[H:8]', inchi_key='XDTMQSROBMDMFD-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651809, cmiles='[H:11][c:1]1[c:2]([c:4]([c:6]([c:5]([c:3]1[H:13])[H:15])[C:7]([H:16])([H:17])[C:8]([H:18])([H:19])[C:9]([H:20])([H:21])[O:10][H:22])[H:14])[H:12]', inchi_key='VAJVDSVGBWFCLW-UHFFFAOYNA-N'), BasicResult(type='basic', record_id=32651781, cmiles='[H:11][c:1]1[c:2]([c:4]([c:6]([c:5]([c:3]1[H:13])[H:15])[C:7]

Each of these collections will store the referenced results in their `entries` dictionary. This dictionary uses the
address of the QCFractal server as keys:

In [7]:
torsion_drive_result_collection.entries.keys()

dict_keys(['https://api.qcarchive.molssi.org:443/'])

This allows results generated by multiple different servers (e.g. a local fractal instance and the public QCArchive
server) to be stored in a single result collection object.

The references to the actual data are then stored in corresponding lists:

In [8]:
torsion_drive_result_collection.entries[qc_client.address][:10]

[TorsionDriveResult(type='torsion', record_id=104348544, cmiles='[H:13][C@@:8]([C:9](=[O:10])[N:17]([H:25])[C@:18]([H:26])([C:19](=[O:20])[N:30]([H:35])[C@:31]([H:36])([C:32](=[O:33])[N:40]([H:42])[C:41]([H:43])([H:44])[H:45])[C:34]([H:37])([H:38])[H:39])[C:21]([H:27])([H:28])[C:22](=[O:23])[O:24][H:29])([C:11]([H:14])([H:15])[H:16])[N:7]([H:12])[C:1](=[O:2])[C:3]([H:4])([H:5])[H:6]', inchi_key='DFRSMLTYXUADOY-WMXHRXQGNA-N'),
 TorsionDriveResult(type='torsion', record_id=104348638, cmiles='[H:13][C@@:8]([C:9](=[O:10])[N:17]([H:25])[C@:18]([H:26])([C:19](=[O:20])[N:31]([H:36])[C@:32]([H:37])([C:33](=[O:34])[N:41]([H:43])[C:42]([H:44])([H:45])[H:46])[C:35]([H:38])([H:39])[H:40])[C:21]([H:27])([H:28])[C:22](=[O:23])[N:24]([H:29])[H:30])([C:11]([H:14])([H:15])[H:16])[N:7]([H:12])[C:1](=[O:2])[C:3]([H:4])([H:5])[H:6]', inchi_key='KYQIKKWSTMGEQA-RIYGDZFYNA-N'),
 TorsionDriveResult(type='torsion', record_id=104348640, cmiles='[H:13][C@@:8]([C:9](=[O:10])[N:17]([H:25])[C@:18]([H:26])([C:19](=[

After running the above command, notice that the entries stored in the collection are not the actual result
records generated and stored on the server, but rather a reference to them. In particular, the unique ID of the record is stored along with a SMILES depiction of the molecule the result was generated for.

The main reason for doing this is that we often would like to be able to state which data we would like to use in
an application without having to create multiple copies of the data. Not only can this take up large amounts of disk space, it runs the risk of data becoming out of sync with the original if the format the records are stored in changes or the local copy of the data is accidentally mutated. Storing a reference to the original data and then retrieving it when needed is typically a cleaner and safer solution.

## Retrieving the result records

The raw result record objects can be easily retrieved using the result collection objects. This allows us to filter the collection to only retrieve the results we want. We'll talk more about filters in an upcoming section, but for now we just apply a SMARTS string to limit our record download to the cysteine record:

In [9]:
from openff.qcsubmit.results.filters import SMARTSFilter

filtered_collection = torsion_drive_result_collection.filter(
    SMARTSFilter(smarts_to_include=["C[SH]"])
)
filtered_collection

TorsionDriveResultCollection(entries={'https://api.qcarchive.molssi.org:443/': [TorsionDriveResult(type='torsion', record_id=104349083, cmiles='[H:30][C@@:24]([C:25](=[O:26])[N:34]([H:41])[C@:35]([H:42])([C:36](=[O:37])[N:50]([H:52])[C:51]([H:53])([H:54])[H:55])[C:38]([H:43])([C:39]([H:44])([H:45])[H:46])[C:40]([H:47])([H:48])[H:49])([C:27]([H:31])([H:32])[S:28][H:33])[N:23]([H:29])[C:9](=[O:10])[C@:8]([H:15])([C:11]([H:16])([C:12]([H:17])([H:18])[H:19])[C:13]([H:20])([H:21])[H:22])[N:7]([H:14])[C:1](=[O:2])[C:3]([H:4])([H:5])[H:6]', inchi_key='NXZIIAGLOSBSES-BCDINFCLNA-N')]}, provenance={'applied-filters': {'SMARTSFilter-0': {'smarts_to_include': ['C[SH]'], 'smarts_to_exclude': None}}}, type='TorsionDriveResultCollection')

Then we can download the actual results. This can be very slow, so it's worth filtering aggressively:

In [10]:
torsion_drive_records = filtered_collection.to_records()
torsion_drive_records

[(TorsiondriveRecord(id=104349083, record_type='torsiondrive', is_service=True, properties={}, extras={}, status=<RecordStatusEnum.complete: 'complete'>, manager_name=None, created_on=datetime.datetime(2022, 5, 31, 3, 11, 41, 558377, tzinfo=datetime.timezone.utc), modified_on=datetime.datetime(2022, 5, 31, 3, 11, 41, 558375, tzinfo=datetime.timezone.utc), owner_user=None, owner_group=None, compute_history_=None, task_=None, service_=None, comments_=None, native_files_=None, specification=TorsiondriveSpecification(program='torsiondrive', optimization_specification=OptimizationSpecification(program='geometric', qc_specification=QCSpecification(program='psi4', driver=<SinglepointDriver.deferred: 'deferred'>, method='b3lyp-d3bj', basis='dzvp', keywords={'maxiter': 200, 'scf_properties': ['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices']}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtoc

QCSubmit seamlessly takes care of pulling the data from the server in the most efficient way making sure to take
advantage of the pagination that QCFractal provides. Further, it attempts to cache all calls to the server so that
multiple calls to `to_records` does not need to constantly query the server.

Notice that not only are the raw result records retrieved, but also an OpenFF `Molecule` object is created for each result record. This molecule has the correct ordering and also stores any conformers associated with the
result collection. For basic collections, the conformer is the one that was used in any calculations; for optimization collections, it is the final conformer yielded by the optimization; and for torsion drives, it is the lowest energy conformer for each sampled torsion angle.

### Inspecting results

In the case of torsion drive records, we can easily iterate over the grid ID, the associated conformer, and the
associated energy in one go:

In [11]:
torsion_drive_record, molecule = torsion_drive_records[0]
for grid_id, qc_conformer in zip(
    molecule.properties["grid_ids"][:10], molecule.conformers
):
    qc_energy = torsion_drive_record.final_energies[grid_id]

    print(f"{grid_id} E={qc_energy:.4f} Ha")

(-165, -165) E=-1546.1642 Ha
(-165, -150) E=-1546.1627 Ha
(-165, -135) E=-1546.1621 Ha
(-165, -120) E=-1546.1617 Ha
(-165, -105) E=-1546.1625 Ha
(-165, -90) E=-1546.1642 Ha
(-165, -75) E=-1546.1666 Ha
(-165, -60) E=-1546.1682 Ha
(-165, -45) E=-1546.1684 Ha
(-165, -30) E=-1546.1689 Ha


### Basic results from optimization results

It is common for certain datasets within a QCFractal server to be created using the output of another dataset. This is especially the case for datasets of hessian records that are computed using the conformer produced by an optimization.

The `OptimizationResultCollection` currently provides a `to_basic_result_collection` method to handle such cases. This can take some time to run:

In [12]:
derived_hessian_collection = optimization_result_collection.to_basic_result_collection(
    driver="hessian"
)
derived_hessian_collection.n_results

2378

This is a particularly useful way to access hessian data contained within older datasets. Older datasets do not usually
store SMILES information for their result records, and hence it can be difficult to know exactly which molecule the
hessian was computed for. The `to_basic_result_collection` method takes care of this by propagating SMILES information
from the parent optimization record down to the child hessian result record.

In addition to retrieving already computed datasets, the optimization result collection provides a utility for
generating a new QC dataset based on the optimized conformers:

In [13]:
from qcportal.singlepoint import SinglepointDriver

hessian_dataset = optimization_result_collection.create_basic_dataset(
    dataset_name="My Dataset",
    description="A dataset created from an optimization result collection.",
    tagline="Contains hessian data.",
    driver=SinglepointDriver.hessian,
)

[03:51:45] WARNING: Charges were rearranged

[03:51:45] WARNING: Charges were rearranged



[03:51:45] WARNING: Charges were rearranged

[03:51:45] WARNING: Charges were rearranged



[03:51:48] WARNING: Charges were rearranged

[03:51:48] WARNING: Charges were rearranged

[03:51:48] WARNING: Charges were rearranged

[03:51:48] WARNING: Charges were rearranged



[03:51:50] WARNING: Charges were rearranged

[03:51:50] WARNING: Charges were rearranged

[03:51:50] WARNING: Charges were rearranged

[03:51:50] WARNING: Charges were rearranged



[03:51:52] WARNING: Charges were rearranged

[03:51:52] WARNING: Charges were rearranged



...

[03:52:24] WARNING: Charges were rearranged

[03:52:24] WARNING: Charges were rearranged



[03:52:25] WARNING: Charges were rearranged

[03:52:25] WARNING: Charges were rearranged

[03:52:25] WARNING: Charges were rearranged

[03:52:25] WARNING: Charges were rearranged



[03:52:38] WARNING: Charges were rearranged

[03:52:38] WARNING: Charges were rearranged



[03:52:39] WARNING: Charges were rearranged

[03:52:39] WARNING: Charges were rearranged

[03:52:39] WARNING: Charges were rearranged

[03:52:39] WARNING: Charges were rearranged



[03:52:41] WARNING: Charges were rearranged

[03:52:41] WARNING: Charges were rearranged



The resulting dataset can then be submitted to a running QCFractal server.

## Filtering result collections

A powerful feature of the result collections is the ability to easily filter the entries it contains using a diverse
range of filters, such as filtering out specific molecules based on SMILES patterns, records where the
connectivity of the molecule changed during the optimization, or much more!

The built-in filters are stored in the `openff.qcsubmit.results.filters` module:

In [14]:
from openff.qcsubmit.results import filters

Let's apply some basic filters to our optimization collection:

In [15]:
from qcportal.record_models import RecordStatusEnum

filtered_collection = optimization_result_collection.filter(
    filters.RecordStatusFilter(status=RecordStatusEnum.complete),
    filters.ConnectivityFilter(tolerance=1.2),
    filters.ElementFilter(
        # The elements supported by OpenFF 1.3.0
        allowed_elements=["H", "C", "N", "O", "S", "P", "F", "Cl", "Br", "I"]
    ),
    filters.ConformerRMSDFilter(max_conformers=10),
)

print("===========")
print(f"N RECORDS INITIAL: {optimization_result_collection.n_results}")
print(f"N RECORDS FINAL:   {filtered_collection.n_results}")

print(f"N MOLECULES INITIAL: {optimization_result_collection.n_molecules}")
print(f"N MOLECULES FINAL:   {filtered_collection.n_molecules}")
print("===========")

N RECORDS INITIAL: 2398
N RECORDS FINAL:   1608
N MOLECULES INITIAL: 419
N MOLECULES FINAL:   419


Here we have removed:

* any incomplete records using the `RecordStatusFilter`

* records whose whereby a connectivity during the computation, e.g. a proton transfer occurred
  using the `ConnectivityFilter`

* records that were computed for molecules composed of elements that are not supported by the current
  OpenFF force fields

and finally, a `ConformerRMSDFilter` was applied. When a collection contains multiple optimized conformers for the
same molecule, the `ConformerRMSDFilter` will only retain up to a maximum number of conformers for that molecule that
are distinct to within a specified RMSD tolerance.

We could have also made use of the `LowestEnergyFilter` to only retain the lowest energy conformer associated with each
unique molecule in the collection.

The filtered result collection will record provenance information about which filters were applied:

In [16]:
filtered_collection.provenance

{'applied-filters': {'RecordStatusFilter-0': {'status': <RecordStatusEnum.complete: 'complete'>},
  'ConnectivityFilter-1': {'tolerance': 1.2},
  'ElementFilter-2': {'allowed_elements': ['H',
    'C',
    'N',
    'O',
    'S',
    'P',
    'F',
    'Cl',
    'Br',
    'I']},
  'ConformerRMSDFilter-3': {'max_conformers': 10,
   'rmsd_tolerance': 0.5,
   'heavy_atoms_only': True,
   'check_automorphs': True}}}

## Additional utilities

In addition to providing an interface for curating collections of QC results, the result collection objects also expose
a number of quality of life utilities for visualizing and analysing the stored results.

A pdf showing the molecules within a result collection can be easily generated:

In [17]:
from openff.toolkit import Molecule

from openff.qcsubmit.utils.visualize import molecules_to_pdf

# We filter by inchi key to make sure that we don't double count molecules
# with different orderings.
unique_smiles = set(
    {
        entry.inchi_key: entry.cmiles
        for entries in energy_result_collection.entries.values()
        for entry in entries
    }.values()
)

molecules = [
    Molecule.from_mapped_smiles(smiles, allow_undefined_stereo=True)
    for smiles in unique_smiles
]

molecules_to_pdf(molecules, "energy-result-collection.pdf", columns=8)

## Advanced visualization

Beyond the basic summary data shown above, we can also directly visualize the torsion drive using interactive `plotly` graphs and the `molecule.visualize("nglview")` function from the OpenFF Toolkit.

In [18]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

torsion_drive_record, molecule = torsion_drive_records[0]
energy_grid = np.zeros((24, 24))
psi_labels = [""] * 24
phi_labels = [""] * 24
for (phi, psi), qc_conformer in zip(
    molecule.properties["grid_ids"], molecule.conformers
):
    qc_energy = torsion_drive_record.final_energies[(phi, psi)]

    phi_bin = int(phi + 165) // 15
    psi_bin = int(psi + 165) // 15
    energy_grid[psi_bin, phi_bin] = qc_energy
    psi_labels[psi_bin] = psi
    phi_labels[phi_bin] = phi


fig = go.FigureWidget(
    data=go.Heatmap(
        z=energy_grid,
        x=phi_labels,
        y=psi_labels,
        colorbar={"title": "Energy (Ha)"},
        hovertemplate="phi: %{x}\npsi: %{y}\nenergy: %{z} Ha",
    ),
    layout=go.Layout(
        title="Val-Ala-Val - central backbone torsiondrive (Ha)",
        xaxis_title="Phi",
        yaxis_title="Psi",
        # autosize=False,
        yaxis_scaleanchor="x",
        xaxis_scaleanchor="y",
    ),
)

view = molecule.visualize("nglview")


def on_click(trace, points, selector):
    print(points)
    for x, y in points.point_inds:
        view.frame = x * 24 + y


heatmap = fig.data[0]
heatmap.on_click(on_click)

container = widgets.GridBox(
    [fig, view],
)
container

GridBox(children=(FigureWidget({
    'data': [{'colorbar': {'title': {'text': 'Energy (Ha)'}},
              '…

## Cached queries

If you provided the `cache_dir` argument to your `PortalClient` initially, providing the same `cache_dir` to future clients
allows them to share the underlying SQLite cache. This `socket.socket` trick temporarily disables access to the network
just to demonstrate that the new calls to `get_dataset` and `BasicResultCollection.from_datasets` can be performed
without new requests to QCArchive.

In [19]:
client = PortalClient("https://api.qcarchive.molssi.org:443", cache_dir=".")

import socket


def guard(*args, **kwargs):
    raise Exception("I told you not to use the Internet!")


old_socket = socket.socket
socket.socket = guard

ds = client.get_dataset("singlepoint", "OpenFF BCC Refit Study COH v1.0")
erc = BasicResultCollection.from_datasets([ds], "spec_2")
assert erc.n_results == energy_result_collection.n_results

socket.socket = old_socket